## 1. Загрузка данных из текстовых файлов
Из файлов 'positive.csv' и 'negative.csv' загружаются тексты положительных и негативных комментариев и производится их предобработка

In [1]:
import pandas as pd
import numpy as np

n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [2]:
import re

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


data = [preprocess_text(t) for t in raw_data]

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=2)

## 2. Метрики качества классификации
Для оценки качества используются точность, полнота и f-мера

In [4]:
from keras import backend as K


def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

Using TensorFlow backend.


## 3. Векторизация слов
Используется алгоритм word2vec, который генерирует векторное представление для каждого слова из текста

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

SENTENCE_LENGTH = 26
NUM = 100000

def get_sequences(tokenizer, x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(x_train)

x_train_seq = get_sequences(tokenizer, x_train)
x_test_seq = get_sequences(tokenizer, x_test)

In [6]:
from gensim.models import Word2Vec
# Загружаем обученную модель
w2v_model = Word2Vec.load('tweets_model.w2v')
DIM = w2v_model.vector_size 
# Инициализируем матрицу embedding слоя нулями
embedding_matrix = np.zeros((NUM, DIM))
# Добавляем NUM=100000 наиболее часто встречающихся слов из обучающей выборки в embedding слой
for word, i in tokenizer.word_index.items():
    if i >= NUM:
        break
    if word in w2v_model.wv.vocab.keys():
        embedding_matrix[i] = w2v_model.wv[word]

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 4. Построение сверточной нейросети 

На вход нейронной сети подается матрица предложения, составленная из векторов отдельных слов. Таким образом, размерность этой матрицы - длина предложения x размернность вектора слова (200 в данном случае)

Для свертки используются фильтры размером 2,3,4,5 слов для параллельной обработки биграмм, триграмм, 4-грамм и 5-грамм соответственно. Для каждой высоты фильтра используется 10 сверточных слоев, функция активации — ReLU.
После обработки слоями свертки карты признаков поступают на слои субдискретизации, где к ним применяется операция 1-max-pooling, тем самым извлекая наиболее значимые n-граммы из текста. На следующем этапе происходит объединение в общий вектор признаков (слой объединения), который подавался в скрытый полносвязный слой с 30 нейронами. На последнем этапе итоговая карта признаков подается на выходной слой нейронной сети с сигмоидальной функцией активации.


In [7]:
from keras.layers import Input
from keras.layers.embeddings import Embedding

tweet_input = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
tweet_encoder = Embedding(NUM, DIM, input_length=SENTENCE_LENGTH,
                          weights=[embedding_matrix], trainable=False)(tweet_input)

In [8]:
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D

branches = []
x = Dropout(0.1)(tweet_encoder)

for size, filters_count in [(2,6), (3, 6)]:
    for i in range(filters_count):
        branch = Conv1D(filters=1, kernel_size=size, padding='valid', activation='sigmoid')(x)
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)

x = concatenate(branches, axis=1)
x = Dropout(0.1)(x)
x = Dense(30, activation='relu')(x)
x = Dense(1)(x)
output = Activation('sigmoid')(x)

model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision, recall, f1])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 26, 200)      20000000    input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 26, 200)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 25, 1)        401         dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

## 5. Обучение и оценка качества

In [9]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("cnn-frozen-embeddings-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', save_best_only=True, mode='max', period=1)
history = model.fit(x_train_seq, y_train, batch_size=32, epochs=10, validation_split=0.25, callbacks = [checkpoint])

Train on 134307 samples, validate on 44769 samples
Epoch 1/10
134307/134307 [==============================] - 86s 643us/step - loss: 0.5805 - precision: 0.6868 - recall: 0.6914 - f1: 0.6801 - val_loss: 0.5166 - val_precision: 0.7501 - val_recall: 0.7307 - val_f1: 0.7338
Epoch 2/10
134307/134307 [==============================] - 82s 609us/step - loss: 0.5293 - precision: 0.7347 - recall: 0.7245 - f1: 0.7221 - val_loss: 0.4996 - val_precision: 0.7487 - val_recall: 0.7527 - val_f1: 0.7443
Epoch 3/10
134307/134307 [==============================] - 80s 599us/step - loss: 0.5172 - precision: 0.7433 - recall: 0.7274 - f1: 0.7280 - val_loss: 0.5016 - val_precision: 0.7175 - val_recall: 0.8067 - val_f1: 0.7541
Epoch 4/10
134307/134307 [==============================] - 79s 586us/step - loss: 0.5107 - precision: 0.7470 - recall: 0.7283 - f1: 0.7305 - val_loss: 0.4919 - val_precision: 0.7540 - val_recall: 0.7589 - val_f1: 0.7503
Epoch 5/10
134307/134307 [==============================] - 79s 5

In [10]:
model.load_weights('cnn-frozen-embeddings-10-0.77.hdf5')

ValueError: You are trying to load a weight file containing 43 layers into a model with 15 layers.

In [ ]:
from sklearn.metrics import classification_report

predicted = np.round(model.predict(x_test_seq))
print(classification_report(y_test, predicted, digits=5))

In [ ]:
from keras import optimizers

model.layers[1].trainable = True
adam = optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[precision, recall, f1])
model.summary()

In [ ]:
checkpoint = ModelCheckpoint("cnn-trainable-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', save_best_only=True, mode='max', period=1)

history_trainable = model.fit(x_train_seq, y_train, batch_size=32, epochs=5, validation_split=0.25, callbacks = [checkpoint])

In [ ]:
plot_history(history_trainable)

In [ ]:
model.load_weights('cnn-trainable-03-0.78.hdf5')

In [ ]:
predicted = np.round(model.predict(x_test_seq))
print(classification_report(y_test, predicted, digits=5))